In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '~/Dropbox/scratch/'

In [3]:
megyelist = ['bácskiskun','baranya','baz','békés','budapest','csongrad','fejer','gyms','hb','heves','jnsz','ke','nograd','pest','somogy','szszb','tolna','vas','veszprem','zala']
megyedict = {'bácskiskun':3,'baranya':2,'baz':5,'békés':4,'budapest':1,'csongrad':6,'fejer':7,'gyms':8,'hb':9,'heves':10,'jnsz':11,'ke':12,'nograd':13,'pest':14,'somogy':15,'szszb':16,'tolna':17,'vas':18,'veszprem':19,'zala':20}

In [4]:
top5_megyek = pd.DataFrame()
top5list = []
for m in megyelist:
    filename = 'top5_' + m + '.csv'
    fullpath = path + filename
    df = pd.read_csv(fullpath,engine='c', header=1, dtype={'Fidesz':np.uint8,'Jobbik':np.uint8,'Hungarian Socialist Party':np.uint8,'Democratic Coalition':np.uint8,'Politics Can Be Different':np.uint8})#,parse_dates=[0])
    df['megye'] = megyedict[m]
    top5list.append(df)
top5_megyek = pd.concat(top5list)

In [5]:
top5_megyek.head()

,Month,Fidesz,Jobbik,Hungarian Socialist Party,Democratic Coalition,Politics Can Be Different,megye
0,2004-01,0,0,0,0,0,3
1,2004-02,0,0,0,0,0,3
2,2004-03,0,0,0,0,0,3
3,2004-04,0,0,77,77,0,3
4,2004-05,0,0,0,0,0,3


In [6]:
small3_megyek = pd.DataFrame()
small3list = []
for m in megyelist:
    filename = 'small3_' + m + '.csv'
    fullpath = path + filename
    df = pd.read_csv(fullpath,engine='c', header=1,dtype={'Fidesz':np.uint8, 'Together':np.uint8, 'Momentum Movement':np.uint8, 'Dialogue for Hungary':np.uint8})
    df['megye'] = megyedict[m]
    small3list.append(df)
small3_megyek = pd.concat(small3list)

In [7]:
small3_megyek.columns

Index(['Month', 'Fidesz', 'Together', 'Momentum Movement',
       'Dialogue for Hungary', 'megye'],
      dtype='object')

In [8]:
m = 'orszagos'
filename = 'top5_' + m + '.csv'
fullpath = path + filename
top5_orszagos = pd.read_csv(fullpath,engine='c', header=1,dtype={'Fidesz':np.uint8,'Jobbik':np.uint8,'Hungarian Socialist Party':np.uint8,'Democratic Coalition':np.uint8,'Politics Can Be Different':np.uint8})
filename = 'small3_' + m + '.csv'
fullpath = path + filename
small3_orszagos = pd.read_csv(fullpath,engine='c', header=1,dtype={'Fidesz':np.uint8, 'Together':np.uint8, 'Momentum Movement':np.uint8, 'Dialogue for Hungary':np.uint8})

In [9]:
top5melt = pd.melt(top5_megyek, id_vars=['Month','megye'], value_vars=['Fidesz','Jobbik','Hungarian Socialist Party','Democratic Coalition','Politics Can Be Different'])
top5melt['value'].astype(int)
top5melt.rename(columns = {'variable':'party','value':'search_intensity'},inplace=True)
top5meltpivot=top5melt.pivot_table(values='search_intensity', index=['Month', 'party'], columns='megye').reset_index()

In [10]:
top5_orszagos_melt = pd.melt(top5_orszagos, id_vars=['Month'], value_vars=['Fidesz','Jobbik','Hungarian Socialist Party','Democratic Coalition','Politics Can Be Different'])
top5_orszagos_melt.rename(columns = {'variable':'party','value':'orszagos'},inplace=True)

In [11]:
small3melt = pd.melt(small3_megyek.drop('Fidesz',axis=1), id_vars=['Month','megye'], value_vars=[ 'Together', 'Momentum Movement', 'Dialogue for Hungary'])
small3melt['value'].astype(int)
small3melt.rename(columns = {'variable':'party','value':'search_intensity'},inplace=True)
small3meltpivot=small3melt.pivot_table(values='search_intensity', index=['Month', 'party'], columns='megye').reset_index()

In [12]:
small3_orszagos_melt = pd.melt(small3_orszagos.drop('Fidesz',axis=1), id_vars=['Month'], value_vars=[ 'Together', 'Momentum Movement', 'Dialogue for Hungary'])
small3_orszagos_melt.rename(columns = {'variable':'party','value':'orszagos'},inplace=True)

In [13]:
all_megye_meltpivot = top5meltpivot.append(small3meltpivot)
all_orszagos_melt = top5_orszagos_melt.append(small3_orszagos_melt)

In [14]:
regressdf = all_megye_meltpivot.merge(all_orszagos_melt,on=['Month', 'party'])

In [39]:
y = regressdf['orszagos']
X = regressdf[[1,          2,          3,          4,
                5,          6,          7,          8,          9,         10,
               11,         12,         13,         14,         15,         16,
               17,         18,         19,         20]]

In [49]:
megyesulyok = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

In [50]:
megyesulyok

array([-342.5520631 ,  274.38510606,  126.15351491,   -1.10597281,
        105.83886981,  750.09537325,   86.55798621,  217.27259296,
        -38.40392642, -504.15556699,  405.53813707, -123.78473604,
        -91.36867771, -400.05228484,  284.02785082, -307.90034991,
       -207.15939294,  229.44657743, -372.94381578,  457.63234367])

In [19]:
all_megye = top5_megyek.merge(small3_megyek.drop('Fidesz',axis=1),on=['Month','megye'])

In [21]:
all_megye['sum_search_index']=all_megye.sum(axis=1,numeric_only=True)-all_megye['megye']

In [24]:
all_megye.columns

Index(['Month', 'Fidesz', 'Jobbik', 'Hungarian Socialist Party',
       'Democratic Coalition', 'Politics Can Be Different', 'megye',
       'Together', 'Momentum Movement', 'Dialogue for Hungary',
       'sum_search_index'],
      dtype='object')

In [54]:
megye_aranyok = pd.concat(copy = False,axis = 1, objs = [all_megye[['megye','Month']],all_megye[['Fidesz', 'Jobbik', 'Hungarian Socialist Party',
       'Democratic Coalition', 'Politics Can Be Different',
       'Together', 'Momentum Movement', 'Dialogue for Hungary']].div(all_megye['sum_search_index'], axis=0).fillna(0)])

In [55]:
megye_aranyok.head()

,megye,Month,Fidesz,Jobbik,Hungarian Socialist Party,Democratic Coalition,Politics Can Be Different,Together,Momentum Movement,Dialogue for Hungary
0,3,2004-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,2004-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2004-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2004-04,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0
4,3,2004-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
all_orszagos = top5_orszagos.merge(small3_orszagos.drop('Fidesz',axis=1),on=['Month'])
all_orszagos['sum_search_index']=all_orszagos.sum(axis=1,numeric_only=True)
orszagos_aranyok = pd.concat(copy = False,axis = 1, objs = [all_orszagos['Month'],all_orszagos[['Fidesz', 'Jobbik', 'Hungarian Socialist Party',
       'Democratic Coalition', 'Politics Can Be Different',
       'Together', 'Momentum Movement', 'Dialogue for Hungary']].div(all_orszagos['sum_search_index'], axis=0).fillna(0)])

In [59]:
orszagos_aranyok.head()

,Month,Fidesz,Jobbik,Hungarian Socialist Party,Democratic Coalition,Politics Can Be Different,Together,Momentum Movement,Dialogue for Hungary
0,2004-01,0.280000,0.280000,0.240000,0.040000,0.0,0.000000,0.040000,0.120000
1,2004-02,0.222222,0.166667,0.055556,0.333333,0.0,0.055556,0.000000,0.166667
2,2004-03,0.357143,0.071429,0.321429,0.071429,0.0,0.071429,0.000000,0.107143
3,2004-04,0.600000,0.088889,0.200000,0.022222,0.0,0.000000,0.022222,0.066667
4,2004-05,0.586207,0.000000,0.310345,0.000000,0.0,0.000000,0.034483,0.068966


In [65]:
orszagos_aranyok.to_csv('GTrend_orszagos_aranyok.csv',index=False)

In [64]:
megye_aranyok.to_csv('GTrend_megye_aranyok.csv',index=False)